# Examen Scala
**Membres du group**
* Mohamed Yansane 216810
* Awa M'baye 219297


**Importation des modules Spark**

In [1]:
//Importation des modules Spark
import $ivy.`org.apache.spark::spark-sql:2.4.5` 
import $ivy.`sh.almond::almond-spark:0.10.9` 
import org.apache.log4j.{Level, Logger}

val rootLogger = Logger.getRootLogger()
rootLogger.setLevel(Level.ERROR)

Logger.getLogger("org.apache.spark").setLevel(Level.WARN)
Logger.getLogger("org.spark-project").setLevel(Level.WARN)

import $ivy.$                                   

import $ivy.$                                

import org.apache.log4j.{Level, Logger}


rootLogger: Logger = org.apache.log4j.spi.RootLogger@4330d875

**Création de la Session Spark**

In [2]:
//Création de la Session Spark
import org.apache.spark.sql.{SparkSession, DataFrame}
import org.apache.spark.sql.types._
import org.apache.spark.sql.functions._

val spark = SparkSession.builder().appName("myApp").master("local").getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
23/05/26 14:07:28 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


import org.apache.spark.sql.{SparkSession, DataFrame}

import org.apache.spark.sql.types._

import org.apache.spark.sql.functions._


spark: SparkSession = org.apache.spark.sql.SparkSession@16fd208b

**Chargement des données (DataFrames)**

In [3]:
//Chargement des données (en DataFrame)
val df_jan = spark.read.format("csv")
  .option("header", "true")
  .load("Sales_January_2019.csv")

val df_feb = spark.read.format("csv")
  .option("header", "true")
  .load("Sales_February_2019.csv")

val df_mar = spark.read.format("csv")
  .option("header", "true")
  .load("Sales_March_2019.csv")

val df_apr = spark.read.format("csv")
  .option("header", "true")
  .load("Sales_April_2019.csv")

val df_may = spark.read.format("csv")
  .option("header", "true")
  .load("Sales_May_2019.csv")

val df_jun = spark.read.format("csv")
  .option("header", "true")
  .load("Sales_June_2019.csv")

val df_jul = spark.read.format("csv")
  .option("header", "true")
  .load("Sales_July_2019.csv")

val df_aug = spark.read.format("csv")
  .option("header", "true")
  .load("Sales_August_2019.csv")

val df_sep = spark.read.format("csv")
  .option("header", "true")
  .load("Sales_September_2019.csv")

val df_oct = spark.read.format("csv")
  .option("header", "true")
  .load("Sales_October_2019.csv")

val df_nov = spark.read.format("csv")
  .option("header", "true")
  .load("Sales_November_2019.csv")

val df_dec = spark.read.format("csv")
  .option("header", "true")
  .load("Sales_December_2019.csv")
//Jonction de tout les dataframes - (combinedDF)
val combinedDF = df_jan.union(df_feb)
                        .union(df_mar)
                        .union(df_apr)
                        .union(df_may)
                        .union(df_jun)
                        .union(df_jul)
                        .union(df_aug)
                        .union(df_sep)
                        .union(df_oct)
                        .union(df_nov)
                        .union(df_dec)

combinedDF.count() 

df_jan: DataFrame = [Order ID: string, Product: string ... 4 more fields]
df_feb: DataFrame = [Order ID: string, Product: string ... 4 more fields]
df_mar: DataFrame = [Order ID: string, Product: string ... 4 more fields]
df_apr: DataFrame = [Order ID: string, Product: string ... 4 more fields]
df_may: DataFrame = [Order ID: string, Product: string ... 4 more fields]
df_jun: DataFrame = [Order ID: string, Product: string ... 4 more fields]
df_jul: DataFrame = [Order ID: string, Product: string ... 4 more fields]
df_aug: DataFrame = [Order ID: string, Product: string ... 4 more fields]
df_sep: DataFrame = [Order ID: string, Product: string ... 4 more fields]
df_oct: DataFrame = [Order ID: string, Product: string ... 4 more fields]
df_nov: DataFrame = [Order ID: string, Product: string ... 4 more fields]
df_dec: DataFrame = [Order ID: string, Product: string ... 4 more fields]
combinedDF: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [Order ID: string, Product: string ... 4 mo

**Encodage des colonnes - Order Date (en timestamp), Price Each (en double), et Quantity (en integer)**

In [4]:
val combinedDF2 = combinedDF.withColumn("Order Date", to_timestamp(col("Order Date"), "MM/dd/yy HH:mm"))
                    .withColumn("Quantity Ordered", col("Quantity Ordered").cast("integer"))
                    .withColumn("Price Each", col("Price Each").cast("double"))


//Verification
combinedDF2.printSchema()
combinedDF2.show(30)


root
 |-- Order ID: string (nullable = true)
 |-- Product: string (nullable = true)
 |-- Quantity Ordered: integer (nullable = true)
 |-- Price Each: double (nullable = true)
 |-- Order Date: timestamp (nullable = true)
 |-- Purchase Address: string (nullable = true)

+--------+--------------------+----------------+----------+-------------------+--------------------+
|Order ID|             Product|Quantity Ordered|Price Each|         Order Date|    Purchase Address|
+--------+--------------------+----------------+----------+-------------------+--------------------+
|  141234|              iPhone|               1|     700.0|2019-01-22 21:25:00|944 Walnut St, Bo...|
|  141235|Lightning Chargin...|               1|     14.95|2019-01-28 14:15:00|185 Maple St, Por...|
|  141236|    Wired Headphones|               2|     11.99|2019-01-17 13:33:00|538 Adams St, San...|
|  141237|    27in FHD Monitor|               1|    149.99|2019-01-05 20:33:00|738 10th St, Los ...|
|  141238|    Wired Head

combinedDF2: DataFrame = [Order ID: string, Product: string ... 4 more fields]

**Question 1 - Quelle année a été la meilleure en termes de ventes ? Combien a-t-on gagné cette
année-là?**                  

In [5]:
// Question 1 : Quelle année a été la meilleure en termes de ventes ? Combien a-t-on gagné cette année-là ?
val salesByYear = combinedDF2.withColumn("Year", year(col("Order Date")))
  .groupBy("Year")
  .agg(sum("Price Each").alias("Total Sales"))
  .orderBy(desc("Total Sales"))

val bestYear = salesByYear.first().getAs[Int]("Year")
val bestYearSales = BigDecimal(salesByYear.first().getAs[Double]("Total Sales"))

println(s"La meilleure année en termes de ventes était  $bestYear avec un total de ventes : $bestYearSales%.2f USD")

La meilleure année en termes de ventes était 2019 avec un total de ventes : 34280627.28 USD


salesByYear: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [Year: int, Total Sales: double]
bestYear: Int = 2019
bestYearSales: BigDecimal = 34280627.280005336

**Question 2 - Quel mois a été le meilleur en termes de ventes ? Combien a-t-on gagné ce mois-là ?**

In [6]:
// Question 2 : Quel mois a été le meilleur en termes de ventes ? Combien a-t-on gagné ce mois-là ?
import java.time.Month
val salesByMonth = combinedDF2.withColumn("Month", month(col("Order Date")))
  .groupBy("Month")
  .agg(sum("Price Each").alias("Total Sales"))
  .orderBy(desc("Total Sales"))

val bestMonth = salesByMonth.first().getAs[Int]("Month")
val bestMonthSales = BigDecimal(salesByMonth.first().getAs[Double]("Total Sales"))
//fonction pour afficher le nom du mois (en Français)
   def afficherMois( a:Int ) : String = {
    a match {
      case 1  => "Janvier"
      case 2  => "Fevrier"
      case 3  => "Mars"
      case 4  => "Avril"
      case 5  => "Mai"
      case 6  => "Juin"
      case 7  => "Juillet"
      case 8  => "Août"
      case 9  => "Septembre"
      case 10 => "Octobre"
      case 11 => "Novembre"
      case 12 => "Decembre"
      case _  => "Invalide"
    }
   }
val bestMonthName = afficherMois(bestMonth)

println(s"Le meilleur mois en termes de ventes était le mois de $bestMonthName avec un total de ventes : $bestMonthSales%.2f USD")


Le meilleur mois en termes de ventes était le mois de Decembre avec un total de ventes : 4588415.41 USD


import java.time.Month

salesByMonth: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [Month: int, Total Sales: double]
bestMonth: Int = 12
bestMonthSales: BigDecimal = 4588415.410001603
defined function afficherMois
bestMonthName: String = "Decembre"

**Question 3 - Quelle ville a enregistré le plus grand nombre de ventes ?**

In [7]:
import org.apache.spark.sql.functions._

// Créez une colonne "City" à partir de la colonne "Purchase Address"
val salesDataWithCity = combinedDF2.withColumn("City", trim(split(col("Purchase Address"), ",")(1)))

// Comptez le nombre de ventes par ville
val salesByCity = salesDataWithCity.groupBy("City").count()

// Trouvez la ville avec le plus grand nombre de ventes
val cityWithMostSalesRow = salesByCity.orderBy(desc("count")).first()
val cityWithMostSales = cityWithMostSalesRow.getAs[String]("City")
val numSales = cityWithMostSalesRow.getAs[Long]("count")

// Affichez la ville avec le plus grand nombre de ventes et le nombre de ventes en chiffres
println(s"La ville avec le plus grand nombre de ventes est : $cityWithMostSales")
println(s"Le nombre de ventes pour cette ville est : $numSales")




La ville avec le plus grand nombre de ventes est : San Francisco
Le nombre de ventes pour cette ville est : 44732


import org.apache.spark.sql.functions._

// Créez une colonne "City" à partir de la colonne "Purchase Address"

salesDataWithCity: DataFrame = [Order ID: string, Product: string ... 5 more fields]
salesByCity: DataFrame = [City: string, count: bigint]
cityWithMostSalesRow: org.apache.spark.sql.Row = [San Francisco,44732]
cityWithMostSales: String = "San Francisco"
numSales: Long = 44732L

**Question 4 - À quelle heure devraient-ils diffuser des publicités pour maximiser les chances que les clients achètent le produit ?**

In [8]:
// Question 4 : À quelle heure devraient-ils diffuser des publicités pour maximiser les chances que les clients achètent le produit ?
val salesByHour = combinedDF2.withColumn("Hour", hour(col("Order Date")))
  .groupBy("Hour")
  .count()
  .orderBy(desc("count"))

val bestHour = salesByHour.first().getAs[Int]("Hour")

println(s"Il est recommandé de diffuser des publicités à $bestHour H pour maximiser les chances d'achat")

Il est recommandé de diffuser des publicités à 19 H pour maximiser les chances d'achat


salesByHour: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [Hour: int, count: bigint]
bestHour: Int = 19

**Question 5 : Quels produits sont le plus souvent vendus ensemble ?**

In [9]:
// Question 5 : Quels produits sont le plus souvent vendus ensemble ?

// Groupez les produits par numéro de commande et collectez les produits dans un tableau
val groupedProducts = combinedDF.groupBy("Order ID").agg(collect_list("Product").as("Products"))

// Filtrez les commandes avec plus d'un produit
val multiProductOrders = groupedProducts.filter(size(col("Products")) > 1)

// Effectuez une jointure sur les commandes pour obtenir les combinaisons de produits
val productCombinations = multiProductOrders.join(multiProductOrders.withColumnRenamed("Products", "Products_1"), Seq("Order ID"))
  .where(size(array_intersect(col("Products"), col("Products_1"))) > 1)
  .select(col("Products").as("Combination"))
  .distinct()

// Affichez les combinaisons de produits les plus souvent vendues ensemble
println("Les produits les plus souvent vendus ensemble :")
productCombinations.show(false)



Les produits les plus souvent vendus ensemble :
+-------------------------------------------------------------------+
|Combination                                                        |
+-------------------------------------------------------------------+
|[27in FHD Monitor, USB-C Charging Cable]                           |
|[Google Phone, 27in FHD Monitor]                                   |
|[Flatscreen TV, 34in Ultrawide Monitor]                            |
|[27in FHD Monitor, AA Batteries (4-pack)]                          |
|[34in Ultrawide Monitor, Macbook Pro Laptop]                       |
|[Bose SoundSport Headphones, Vareebadd Phone]                      |
|[iPhone, 34in Ultrawide Monitor]                                   |
|[USB-C Charging Cable, Macbook Pro Laptop]                         |
|[27in FHD Monitor, AAA Batteries (4-pack)]                         |
|[20in Monitor, 34in Ultrawide Monitor]                             |
|[Apple Airpods Headphones, 27in FHD Monit

groupedProducts: DataFrame = [Order ID: string, Products: array<string>]
multiProductOrders: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [Order ID: string, Products: array<string>]
productCombinations: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [Combination: array<string>]

**Question 6 : Quel produit s'est le plus vendu ? Pourquoi pensez-vous qu'il se soit autant vendu ? (Réponse A)**

In [10]:
// Question 6 : Quel produit s'est le plus vendu ? Pourquoi pensez-vous qu'il se soit autant vendu ?
val bestSellingProduct = combinedDF2.groupBy("Product")
  .agg(sum("Quantity Ordered").alias("Total Quantity"))
  .orderBy(desc("Total Quantity"))
  .first()
  .getAs[String]("Product")

println(s"Le produit le plus vendu est: $bestSellingProduct")

Le produit le plus vendu est: AAA Batteries (4-pack)


bestSellingProduct: String = "AAA Batteries (4-pack)"

**Question 6 (Réponse B)** - La raison pour laquelle les piles AAA se sont autant vendu est parce que la plus part des produit sur le catalogue nécessitent des batteries pour fonctionner

**Question 7 : Quelle est la probabilité que les prochains clients commandent un câble de chargement USB-C ?**

In [11]:
// Question 7 : Quelle est la probabilité que les prochains clients commandent un câble de chargement USB-C ?
val totalOrders = combinedDF2.select("Order ID").distinct().count()
val usbCOrders = combinedDF2.filter(col("Product").contains("USB-C Charging Cable")).count()
val usbCProbability = usbCOrders.toDouble / totalOrders.toDouble

println(s"La probabilité que les prochains clients commandent un câble de chargement USB-C est de $usbCProbability")

La probabilité que les prochains clients commandent un câble de chargement USB-C est de 0.1227478297905727


totalOrders: Long = 178439L
usbCOrders: Long = 21903L
usbCProbability: Double = 0.1227478297905727

**Question 8 : Quelle est la probabilité que les prochains clients commandent un iPhone ?**

In [12]:
// Question 8 : Quelle est la probabilité que les prochains clients commandent un iPhone ?
val totalOrders = combinedDF2.select("Order ID").distinct().count()
val iPhoneOrders = combinedDF2.filter(col("Product").contains("iPhone")).count()
val iPhoneProbability = iPhoneOrders.toDouble / totalOrders.toDouble

println(s"La probabilité que les prochains clients commandent un iPhone est de $iPhoneProbability")

La probabilité que les prochains clients commandent un iPhone est de 0.03834363564019076


totalOrders: Long = 178439L
iPhoneOrders: Long = 6842L
iPhoneProbability: Double = 0.03834363564019076

**Question 9 : Quelle est la probabilité que les prochains clients commandent un téléphone Google ?**

In [13]:
// Question 9 : Quelle est la probabilité que les prochains clients commandent un téléphone Google ?
val totalOrders = combinedDF2.select("Order ID").distinct().count()
val googlePhoneOrders = combinedDF2.filter(col("Product").contains("Google Phone")).count()
val googlePhoneProbability = googlePhoneOrders.toDouble / totalOrders.toDouble

println(s"La probabilité que les prochains clients commandent un téléphone Google est de $googlePhoneProbability")

La probabilité que les prochains clients commandent un téléphone Google est de 0.030962962132717622


totalOrders: Long = 178439L
googlePhoneOrders: Long = 5525L
googlePhoneProbability: Double = 0.030962962132717622

**Question 10 - Quelle est la probabilité que d'autres personnes commandent des écouteurs filaires ?**

In [14]:
// Question 9 : Quelle est la probabilité que les prochains clients commandent des écouteurs filaires ?
val totalOrders = combinedDF2.select("Order ID").distinct().count()
val wiredHeadphoneOrders = combinedDF2.filter(col("Product").contains("Wired Headphones")).count()
val wiredHeadphoneProbability = wiredHeadphoneOrders.toDouble / totalOrders.toDouble

println(s"La probabilité que les prochains clients commandent des écouteurs filaires est de $wiredHeadphoneProbability")

La probabilité que les prochains clients commandent des écouteurs filaires est de 0.10581767438732564


totalOrders: Long = 178439L
wiredHeadphoneOrders: Long = 18882L
wiredHeadphoneProbability: Double = 0.10581767438732564

**Question - Pratique**

Cas d'utilisation concret de Machine Learning :
Un cas d'utilisation concret pour ces données de ventes pourrait consister à créer un modèle prédictif pour estimer les ventes futures d'un produit spécifique. Par exemple, supposons que vous souhaitiez prédire les ventes mensuelles d'un produit donné sur la base des données historiques de ventes. Vous pourriez utiliser un modèle de régression pour cela.

Voici comment vous pourriez procéder :

* Préparez les données : Sélectionnez les variables pertinentes pour votre modèle, telles que le mois, le produit et les ventes. Vous devrez peut-être effectuer des transformations supplémentaires, telles que l'encodage des variables catégorielles.

* Divisez les données : Divisez votre ensemble de données en un ensemble d'entraînement et un ensemble de test. Vous utiliserez l'ensemble d'entraînement pour entraîner votre modèle et l'ensemble de test pour évaluer sa performance.

* Choisissez et entraînez le modèle : Sélectionnez un modèle de régression approprié, tel qu'une régression linéaire ou une régression logistique. Entraînez le modèle en ajustant les coefficients aux données d'entraînement.

* Évaluez le modèle : Utilisez l'ensemble de test pour évaluer les performances de votre modèle. Calculez des métriques telles que l'erreur quadratique moyenne (RMSE) ou le coefficient de détermination (R²) pour évaluer à quel point votre modèle prédit les ventes réelles.

* Faites des prédictions : Une fois que vous êtes satisfait des performances de votre modèle, vous pouvez l'utiliser pour faire des prédictions sur les ventes futures. Par exemple, vous pourriez estimer les ventes mensuelles pour les prochains mois en utilisant les données actuelles.

Il convient de noter que la construction d'un modèle prédictif robuste nécessite souvent plus de données et des techniques plus avancées. Cependant, cet exemple fournit une base pour commencer à explorer l'utilisation de l'apprentissage automatique pour résoudre des problèmes basés sur les données de ventes.